## Objectives




*   Marking all launch sites on a map
*   Marking the success/failed launches for each site on the map
*   Calculating the distances between a launch site to its proximities



In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label



# Create a map centered at a specific location (e.g., NASA's coordinate)
nasa_coordinate = [28.5721, -80.6480]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define the launch sites and their coordinates
launch_sites = [
    {"name": "CCAFS LC-40", "coordinate": [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "coordinate": [28.563197, -80.576820]},
    {"name": "KSC LC-39A", "coordinate": [28.573255, -80.646895]},
    {"name": "VAFB SLC-4E", "coordinate": [34.632834, -120.610745]},
]

# Add circles and markers for each launch site
for launch_site in launch_sites:
    # Add a Circle for the launch site with a popup label
    folium.Circle(
        location=launch_site["coordinate"],
        radius=1000,  # Adjust the radius as needed
        color='#000000',
        fill=True,
    ).add_child(folium.Popup(launch_site["name"])).add_to(site_map)

    # Add a Marker for the launch site with a label
    folium.map.Marker(
        location=launch_site["coordinate"],
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{launch_site["name"]}</b></div>'
        ),
    ).add_to(site_map)

# Display the map
site_map.save("launch_sites_map.html")
display(site_map)


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [16]:
marker_cluster = MarkerCluster()


In [17]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def get_marker_color(class_value):
    return 'green' if class_value == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
    )
    marker_cluster.add_child(marker)

site_map

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [26]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [27]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [28]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
# Coordinates of the launch site (replace with your launch site's coordinates)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates of the closest coastline point (replace with your selected coordinates)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the closest coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a Marker for the distance between coastline and launch site
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the distance_marker to the map
site_map.add_child(distance_marker)

In [29]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Draw a PolyLine between the launch site and the closest coastline point
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1,
)
site_map.add_child(lines)

# Display the map with the MousePosition, distance marker, and PolyLine
site_map

In [25]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [30]:
# Coordinates of the launch site (replace with your launch site's coordinates)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates of the closest city, railway, highway, etc. (replace with your selected coordinates)
selected_point_lat = 28.550550
selected_point_lon = -80.572900  # Replace with the actual coordinates

# Calculate the distance between the launch site and the selected point
distance_to_point = calculate_distance(launch_site_lat, launch_site_lon, selected_point_lat, selected_point_lon)

# Create a Marker for the distance to the selected point
distance_marker = folium.Marker(
    location=[selected_point_lat, selected_point_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_point),
    )
)

# Add the distance_marker to the map
site_map.add_child(distance_marker)

# Draw a PolyLine between the launch site and the selected point
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [selected_point_lat, selected_point_lon]],
    weight=1,
)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map with the MousePosition, distance marker, and PolyLine
site_map
